In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

from datasets import load_dataset
dataset = load_dataset('imdb')
texts = dataset['train']['text']
labels = dataset['train']['label']

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
small_model_names = [
    'distilbert-base-uncased',
    'textattack/bert-base-uncased-imdb',
    'bert-base-uncased',
    'facebook/bart-base'
]

model_classes = {model_name: (AutoTokenizer, AutoModelForSequenceClassification) for model_name in small_model_names}

epochs = 3
accuracy_arr = {model_name: [] for model_name in small_model_names}
precision_arr = {model_name: [] for model_name in small_model_names}
recall_arr = {model_name: [] for model_name in small_model_names}
f1_arr = {model_name: [] for model_name in small_model_names}

for model_name in small_model_names:

    tokenizer_class, model_class = model_classes[model_name]
    
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name, num_labels=2)


    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

    train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
    val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))


    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()


        model.eval()
        predictions = []
        true_labels = []
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = batch
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                _, predicted_labels = torch.max(outputs.logits, 1)

                predictions.extend(predicted_labels.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())


        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions)
        recall = recall_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions)


        accuracy_arr[model_name].append(accuracy)
        precision_arr[model_name].append(precision)
        recall_arr[model_name].append(recall)
        f1_arr[model_name].append(f1)

        print(f"Model: {model_name} - Epoch {epoch + 1}/{epochs} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


for model_name in small_model_names:
    print(f"\nFinal Scores for {model_name}:")
    print(f"Accuracy: {accuracy_arr[model_name]}")
    print(f"Precision: {precision_arr[model_name]}")
    print(f"Recall: {recall_arr[model_name]}")
    print(f"F1: {f1_arr[model_name]}")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\Pratham\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pratham\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np



normalized_accuracy = np.zeros(len(small_model_names))
normalized_precision = np.zeros(len(small_model_names))
normalized_recall = np.zeros(len(small_model_names))
normalized_f1 = np.zeros(len(small_model_names))


for i, model_name in enumerate(small_model_names):
    accuracy_values = np.array(accuracy_arr[model_name])
    precision_values = np.array(precision_arr[model_name])
    recall_values = np.array(recall_arr[model_name])
    f1_values = np.array(f1_arr[model_name])


    normalized_accuracy[i] = accuracy_values.mean() / accuracy_values.std()
    normalized_precision[i] = precision_values.mean() / precision_values.std()
    normalized_recall[i] = recall_values.mean() / recall_values.std()
    normalized_f1[i] = f1_values.mean() / f1_values.std()


geometric_mean = np.sqrt(normalized_accuracy * normalized_precision * normalized_recall * normalized_f1)

model_ranking = np.argsort(geometric_mean)[::-1] + 1  

print("\nTOPSIS Scores:")
for i, model_name in enumerate(small_model_names):
    print(f"{model_name}: {geometric_mean[i]:.4f} (Rank: {model_ranking[i]})")


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar(small_model_names, geometric_mean, color='skyblue')
plt.xlabel('Models')
plt.ylabel('TOPSIS Score')
plt.title('TOPSIS Scores for Text Classification Models')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
